In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
from functools import partial

from optimization import jg_opt
from pisa.administrative_area import AdministrativeArea
from pisa.facilities import Facilities
from pisa.population import WorldpopPopulation
from pisa.population_served_by_isopolygons import get_population_served_by_isopolygons
from pisa.visualisation import (
    plot_facilities,
    plot_isochrones,
    plot_population,
    plot_population_heatmap,
)



In [18]:
import os
from dotenv import load_dotenv

# load environment variables from an `.env` file in the local root directory
load_dotenv()

CBC_SOLVER_PATH = os.getenv('CBC_SOLVER_PATH')   # path to the cbc executable (e.g. /opt/homebrew/bin/cbc)


### Define Administrative Area

Let op: the administrative area is a country because of the package that we use for fetching the data (gadm).

The naming is confusing: we want the administrative area to be a subset of a country (in this case, the administrative area should be Baucau). 

Should be easy to fix once we change gadm to pygadm (see issue #59 on GitHub)

In [3]:
timor_leste = AdministrativeArea(country_name="Timor-Leste", admin_level=1)

# these are the boundaries of Baucau
# type: Polygon
baucau = timor_leste.get_admin_area_boundaries("Baucau")

INFO:pisa.administrative_area:Validating country name: Timor-Leste
INFO:pisa.administrative_area:Country name 'Timor-Leste' validated successfully
INFO:pisa.administrative_area:Retrieving boundaries of all administrative areas of level 1 for country Timor-Leste


### Facilities

#### Get existing facilities (in our case, hospitals) from OSM

In [4]:
# Suppress user warning about geometry in geographic CRS. Centroid is calculated over a single facility (e.g. a hospital), 
# so distances are very small and projection isn't necessary
warnings.filterwarnings('ignore', message="Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect", category=UserWarning)


hospitals_df = Facilities(admin_area_boundaries=baucau).get_existing_facilities()



INFO:pisa.facilities:Retrieving existing facilities with tags {'amenity': 'hospital', 'building': 'hospital'} using OSM.


INFO:pisa.facilities:Successfully retrieved existing facilities from OSM.


### **HOTFIX**: 
remove next cell when renaming index of hospitals_df is fixed

In [5]:
hospitals_df.reset_index(drop=False, inplace=True)

#### Estimate potential locations for new facilities


In [6]:
potential_hospitals_df = Facilities(admin_area_boundaries=baucau).estimate_potential_facilities(spacing=0.05)

### Get population 
In this example, from WorldPop

In [7]:
# TODO: why "ID" column? extra index for what reason? Maybe becomes apparent later?

population_gdf = WorldpopPopulation(
    admin_area_boundaries=baucau, iso3_country_code=timor_leste.get_iso3_country_code()
).get_population_gdf()

population_gdf.head()

,ID,longitude,latitude,population,geometry
0,0,126.14917,-8.67333,0.10,POINT (126.14917 -8.67333)
1,1,126.14917,-8.67250,0.09,POINT (126.14917 -8.6725)
2,2,126.15000,-8.67333,0.10,POINT (126.15 -8.67333)
3,3,126.15000,-8.67250,0.09,POINT (126.15 -8.6725)
4,4,126.15000,-8.67167,0.09,POINT (126.15 -8.67167)


### Calculate isopolygons

Here we make some choices:
- distance type
- distance values
- mode of transport 

Valid values for constants are in the script pisa.constants

In [8]:
DISTANCE_TYPE = "length"

DISTANCE_VALUES = [2000, 5000, 10000]

MODE_OF_TRANSPORT = "driving" 

#### Using OSM

For this, we need to get the road network from osmnx

In [9]:
from pisa.osm_road_network import OsmRoadNetwork

road_network = OsmRoadNetwork(
    admin_area_boundaries=baucau, 
    mode_of_transport=MODE_OF_TRANSPORT, 
    distance_type=DISTANCE_TYPE
).get_osm_road_network()

INFO:pisa.osm_road_network:OSM road network set with parameters network_type 'drive' and distance_type 'length'


#### Calculate isopolygons for existing facilities

In [10]:
from pisa.isopolygons import OsmIsopolygonCalculator

isopolygons_existing_facilities = OsmIsopolygonCalculator(
    facilities_df=hospitals_df, 
    distance_type=DISTANCE_TYPE, 
    distance_values=DISTANCE_VALUES, 
    road_network=road_network
).calculate_isopolygons()

/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/isopolygons.py:220: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  disks = nodes_gdf.buffer(node_buffer).geometry
/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/isopolygons.py:225: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cylinders = edges_gdf.buffer(edge_buffer).geometry
/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/isopolygons.py:220: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  disks = nodes_gdf.buffer(node_buffer).geometry
/Users/catherine/code/Public-Infrastructure-Serv

#### Calculate isopolygons for potential facilities

In [11]:
isopolygons_potential_facilities = OsmIsopolygonCalculator(
    facilities_df=potential_hospitals_df, 
    distance_type=DISTANCE_TYPE, 
    distance_values=DISTANCE_VALUES, 
    road_network=road_network
).calculate_isopolygons()

/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/isopolygons.py:220: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  disks = nodes_gdf.buffer(node_buffer).geometry
/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/isopolygons.py:225: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cylinders = edges_gdf.buffer(edge_buffer).geometry
/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/isopolygons.py:220: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  disks = nodes_gdf.buffer(node_buffer).geometry
/Users/catherine/code/Public-Infrastructure-Serv

### **HOTFIX!!**

Eliminate next 2 cells when index in OsmIsopolygonCalculator and OsmIsopolygonCalculatorAlternative are fixed

In [ ]:
isopolygons_existing_facilities.reset_index(drop=True, inplace=True)

isopolygons_existing_facilities.head(2)

,ID_2000,ID_5000,ID_10000
0,POLYGON ((126.59878421714033 -8.54764218472667...,POLYGON ((126.59761276961231 -8.57116537023302...,POLYGON ((126.56068305448515 -8.56751807760911...
1,POLYGON ((126.65907104033573 -8.62868878467725...,POLYGON ((126.6311023795325 -8.646327183432366...,POLYGON ((126.63106042126435 -8.64641589673682...


In [13]:
isopolygons_potential_facilities.reset_index(drop=True, inplace=True)

isopolygons_potential_facilities.head()

,ID_2000,ID_5000,ID_10000
0,POLYGON ((126.2186653852804 -8.669072990322016...,POLYGON ((126.22078733981135 -8.69324047794638...,POLYGON ((126.22232917023302 -8.69708446961230...
1,POLYGON ((126.2186653852804 -8.669072990322016...,POLYGON ((126.21992567953251 -8.68224218343236...,POLYGON ((126.22232917023302 -8.69708446961230...
2,POLYGON ((126.2760704852804 -8.591701490322016...,POLYGON ((126.3081564852804 -8.592539190322016...,POLYGON ((126.2926742743849 -8.608415374758932...
3,POLYGON ((126.2100648852804 -8.582364690322017...,POLYGON ((126.2100648852804 -8.582364690322017...,"POLYGON ((126.1913314 -8.5236606, 126.19123338..."
4,POLYGON ((126.1936728943455 -8.525232959384992...,"POLYGON ((126.1913314 -8.5236606, 126.19123338...",POLYGON ((126.1936728943455 -8.525232959384992...


## Prepare optimization data

Preparing the variables that go into the cell below

In [15]:
population_served_current = get_population_served_by_isopolygons(
    grouped_population=population_gdf,
    isopolygons=isopolygons_existing_facilities
)

current = {DISTANCE_TYPE: population_served_current}

In [16]:
population_served_potential = get_population_served_by_isopolygons(
        grouped_population=population_gdf,
        isopolygons=isopolygons_potential_facilities
)

potential = {DISTANCE_TYPE: population_served_potential}

In [17]:
population_count = population_gdf.population.values

In [19]:

BUDGET = [5, 20, 50] # budget for the optimization in terms of how many locations can be built

cbc_optimize = partial(jg_opt.OpenOptimize, solver_path=CBC_SOLVER_PATH)

values, solutions = jg_opt.Solve(
    household=population_count,
    current=current,
    potential=potential,
    accessibility=DISTANCE_TYPE,
    budgets=BUDGET,
    optimize=cbc_optimize,
    type="ID",
)

In [20]:
values

,10000,5000,2000
5,0.388286,0.256445,0.126768
20,0.392914,0.266618,0.134405
50,0.392914,0.268646,0.135822


In [21]:
solutions

,10000,5000,2000
5,"[25, 27, 43, 10, 17]","[27, 22, 25, 9, 43]","[39, 43, 25, 7, 22]"
20,"[25, 27, 43, 10, 17, 18, 22, 35, 36, 14, 38, 0...","[27, 22, 25, 9, 43, 16, 18, 42, 50, 38, 39, 28...","[39, 43, 25, 7, 22, 27, 24, 34, 26, 13, 32, 10..."
50,"[25, 27, 43, 10, 17, 18, 22, 35, 36, 14, 38, 0...","[27, 22, 25, 9, 43, 16, 18, 42, 50, 38, 39, 28...","[39, 43, 25, 7, 22, 27, 24, 34, 26, 13, 32, 10..."
